<a href="https://colab.research.google.com/github/intelbarna97/neural-network-with-optimizers/blob/main/AMBPZ9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import math, random, time

from sklearn.metrics import mean_squared_error

In [ ]:
class DataTransform:
  def __init__(self, row_start, row_end, history_length, source, delimiter=","):
    self.data = []
    self.start = row_start
    self.end = row_end
    self.history_length = history_length
    self.source = source
    self.delimiter=delimiter

  def is_number(self, s):
    try:
        float(s)
        return True
    except ValueError:
        return False

  def reader(self):
    self.data = [sor.strip().split(sep=self.delimiter) for sor in open(self.source)]

  def process(self):

    self.reader()
    self.data=[[float(adat) if self.is_number(adat) else adat  for adat in sor[self.start-1:self.end] ] for sor in self.data[1:]]

    split = self.split()
    reshape = self.reshape(split[0], split[2])

    return reshape[0], reshape[1], split[1], split[3]

  def split(self):
    train = self.data[:round(len(self.data)*0.75)]
    test = self.data[round(len(self.data)*0.75):]
    extreme = self.extreme()
    min, max = extreme[0], extreme[1]


    train_x = (np.array([ train[i:i+self.history_length] for i in range(len(train)-self.history_length)])+min)/(min+max)
    train_y = (np.array([ train[i+self.history_length] for i in range(len(train)-self.history_length)])+min)/(min+max)
    test_x = (np.array([ test[i:i+self.history_length] for i in range(len(test)-self.history_length)])+min)/(min+max)
    test_y = (np.array([ test[i+self.history_length] for i in range(len(test)-self.history_length)])+min)/(min+max)

    return train_x, train_y, test_x, test_y

  def extreme(self):
    data = np.array(self.data)
    return min(data.flatten())<0 if abs(min(data.flatten())) else 0, abs(max(data.flatten()))+1

  def reshape(self, train, test):
    reshaped_train_x = train.reshape(train.shape[0],train.shape[1]*train.shape[2])
    reshaped_test_x = test.reshape(test.shape[0],test.shape[1]*test.shape[2])
    return reshaped_train_x, reshaped_test_x




In [ ]:
class Optimizers:

  def __init__(self, weight_list, optimizer_name, learning_rate):
    self.weight_list = weight_list
    self.optimizer_name = optimizer_name
    self.ml=[np.zeros_like(i) for i in weight_list]
    self.vl=[np.zeros_like(i) for i in weight_list]
    self.learning_rate = learning_rate

  def adam(self, grad, l, epoch, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=0.00000001):
    self.ml[l] = beta1 * self.ml[l] + (1-beta1)*grad
    self.vl[l] = beta2 * self.vl[l] + (1-beta2)*np.power(grad,2)
    mh = self.ml[l]/(1-np.power(beta1,epoch+1))
    vh = self.vl[l]/(1-np.power(beta2,epoch+1))
    self.weight_list[l] += learning_rate*mh/(np.sqrt(vh)+epsilon)
    return self.weight_list

  def nadam(self, grad, l, epoch, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=0.00000001):
    self.ml[l] = beta1 * self.ml[l] + (1-beta1)*grad
    self.vl[l] = beta2 * self.vl[l] + (1-beta2)*np.power(grad,2)
    mh = self.ml[l]/(1-np.power(beta1,epoch+1)) + (1 - beta1)*grad/(1 - np.power(beta1,epoch+1))
    vh = self.vl[l]/(1-np.power(beta2,epoch+1))
    self.weight_list[l] += learning_rate*mh/(np.sqrt(vh)+epsilon)
    return self.weight_list

  def simple(self, l, grad, learning_rate=0.03):
    self.weight_list[l] += learning_rate * grad
    return self.weight_list

  def momentum(self, l, grad, learning_rate=0.03):
    self.weight_list[l] += 0.1*self.ml[l]
    self.ml[l][:]=learning_rate * grad
    self.weight_list[l] += 0.9*self.ml[l]
    return self.weight_list


  def execute(self, l, delta, neurons, neuronlayer_list, epoch):

    grad = delta[l].reshape((neurons[l+1],1))*neuronlayer_list[l].reshape((1,neurons[l]))

    if(self.optimizer_name == "adam"):

      return self.adam(grad, l, epoch, self.learning_rate)

    elif(self.optimizer_name == "nadam"):

      return self.nadam(grad, l, epoch, self.learning_rate)

    elif(self.optimizer_name == "simple"):

      return self.simple(l, grad, self.learning_rate)

    elif(self.optimizer_name == "momentum"):

      return self.momentum(l, grad, self.learning_rate)

    else:

      return self.simple(l, grad, self.learning_rate)


In [ ]:
class mlp:

  def activation_tanh(self, x):     return np.tanh(x)
  def dactivation_tanh(self, x):    return 1.0 - x**2
  def activation_GCU(self, x):      return x*np.cos(x)
  def dactivation_GCU(self, x):     return np.cos(x)-x*np.sin(x)
  def activation_sigmoid(self, x):  return 1.0/(1.0 + np.exp(-x))
  def dactivation_sigmoid(self, x): return x*(1.0-x)


  def __init__(self, input, output, test_input, test_output, acti_name, layer1, layer2, learning_rate, optimizer_name):
    self.input = input
    self.output = output
    self.bias = 1
    self.sumerr = len(input)
    self.input_test = test_input
    self.output_test = test_output


    if(acti_name == "tanh"):
      self.acti = self.activation_tanh
      self.dacti = self.dactivation_tanh
    elif(acti_name == "GCU"):
      self.acti = self.activation_GCU
      self.dacti = self.dactivation_GCU
    elif(acti_name == "sigmoid"):
      self.acti = self.activation_GCU
      self.dacti = self.dactivation_GCU
    else:
      self.acti = self.activation_tanh
      self.dacti = self.dactivation_tanh

    self.neurons=[len(self.input[0])+self.bias, layer1, layer2, len(self.output[0])]

    self.weighted_list = [np.random.random((self.neurons[l+1], self.neurons[l]))*0.4-0.2 for l in range(len(self.neurons)-1)]

    self.delta = [np.zeros((self.neurons[l+1])) for l in range(len(self.neurons)-1)]

    self.optimizer = Optimizers(self.weighted_list,optimizer_name, learning_rate)


  def backward_propagation(self, output, neuronlayer_list, epoch):
    error = output - neuronlayer_list[-1]

    for l in reversed(range(len(self.neurons)-1)):
            if l == len(self.neurons)-2:
                self.delta[l][:] = error*self.dacti(neuronlayer_list[-1])
            else:
                np.dot(self.delta[l+1],self.weighted_list[l+1], out=self.delta[l])
                self.delta[l] *= self.dacti(neuronlayer_list[l+1])
            self.weighted_list = self.optimizer.execute(l, self.delta, self.neurons, neuronlayer_list, epoch)
    return error


  def testing(self):
    sumerr = 0.0
    compare=[]
    for inp, out in zip(self.input_test, self.output_test):
        nl = [ np.array(list(inp) + [1.0]*self.bias) ]
        for l in range(len(self.neurons)-1):
            nl.append(self.acti(np.dot(self.weighted_list[l],nl[l])))
        compare.append(nl[-1])
    compare=np.array(compare)
    print('mean squared error :', mean_squared_error(self.output_test, compare))



  def fit(self, epoch_range=20):

    if(epoch_range<1):
      epoch_range=20
    epoch = 0

    time1=time.time()
    asd=[]
    while(self.sumerr/len(self.input)>=0.00001 and epoch<epoch_range):
      self.sumerr=0.0
      epoch+=1
      time2=time.time()

      for input, output in zip(self.input, self.output):
        neuronlayer_list=[np.array(list(input) + [1.0]*self.bias)]

        for l in range(len(self.neurons)-1):
          neuronlayer_list.append(self.acti(np.dot(self.weighted_list[l],neuronlayer_list[l])))

        backward = self.backward_propagation(output, neuronlayer_list, epoch)
        error = backward
        self.sumerr += sum( [error[j]**2 for j in range(self.neurons[-1])])
      print ("iter #"+str(epoch),self.sumerr/len(self.input), round(time.time()-time2,3))
      break
    print ("Summary: ",self.sumerr/len(self.input), round(time.time()-time1,3))
    self.testing()



In [ ]:
data = DataTransform(2, 6, 3, "/content/AAPL.csv")
processed_data = data.process()
print(processed_data[0].shape,
processed_data[1].shape,
processed_data[2].shape,
processed_data[3].shape,
len(processed_data[0]))


(8061, 15) (2685, 15) (8061, 5) (2685, 5) 8061


In [ ]:


test = mlp(processed_data[0], processed_data[2], processed_data[1], processed_data[3], "tanh", 10, 10, 0.003, "adam")

test.fit(20)


iter #1 4.2736187373303426e-06 1.374
Summary:  4.2736187373303426e-06 1.376
mean squared error : 0.07824879291172444
